## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob

## Read-in

In [2]:
# Get today's date
today = datetime.now()

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Jul. 1st Jul. 8th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [8]:
# # Filter dates between. If only using data looking back one week, this shouldn't change the df
# df = df[(df['SOLD DATE'] >= '2024-06-10') & (df['SOLD DATE'] <= '2024-06-17')]

In [7]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
105,PAST SALE,2024-07-07,Condo/Co-op,2924 Collins Ave #203,Miami Beach,FL,33140.0,285000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/2924-Col...,MARMLS,A11500005,N,Y,25.805583,-80.125005
75,PAST SALE,2024-07-05,Condo/Co-op,8025 NW 104th Ave #01,Doral,FL,33178.0,497000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Doral/8025-NW-104th-...,MARMLS,A11495115,N,Y,25.846712,-80.364590
35,PAST SALE,2024-07-05,Condo/Co-op,3116 Bird Ave #3116,Miami,FL,33133.0,475000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3116-Bird-Ave-...,MARMLS,A11567431,N,Y,25.735049,-80.244102
97,PAST SALE,2024-07-05,Condo/Co-op,8365 SW 152nd Ave Unit C-208,Miami,FL,33193.0,230000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/8365-SW-152nd-...,MARMLS,A11511926,N,Y,25.688575,-80.438462
33,PAST SALE,2024-07-05,Condo/Co-op,2750 NE 183rd St #2304,Aventura,FL,33160.0,245000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/2750-NE-183...,MARMLS,A11565543,N,Y,25.943983,-80.144896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,PAST SALE,2024-07-01,Condo/Co-op,200 Biscayne Boulevard Way #3805,Miami,FL,33131.0,640000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/200-Biscayne-B...,MARMLS,A11545329,N,Y,25.770430,-80.189163
77,PAST SALE,2024-07-01,Condo/Co-op,1203 Marseille Dr #22,Miami Beach,FL,33141.0,160000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1203-Mar...,MARMLS,A11520121,N,Y,25.856568,-80.131662
78,PAST SALE,2024-07-01,Condo/Co-op,5390 W 21st Ct #108,Hialeah,FL,33016.0,255000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hialeah/5390-W-21st-...,MARMLS,A11542181,N,Y,25.871216,-80.326157
80,PAST SALE,2024-07-01,Condo/Co-op,1200 Brickell Bay Dr #2209,Miami,FL,33131.0,740000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1200-Brickell-...,MARMLS,A11520060,N,Y,25.761769,-80.189921


In [8]:
df = df.sort_values(by='PRICE',ascending=False)
df_top_ten = df.head(10)

## Current Week's Values

In [9]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

118


In [10]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$780,998


In [11]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$494


In [12]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$92,157,794


In [13]:
# # Define the date range
# start_date = '2024-05-20'
# end_date = '2024-05-26'

# # Filter rows where 'SOLD DATE' is within the specified range
# second_week_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

## Previous Week's Values

In [14]:
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

Input Previous Week Condo Sales Total (number units sold):
147
Input Previous Week Condo Average Sales Price:
748529
Input Previous Week Condo Average PSF:
517
Input Previous Week Condo Sales total (ex: 198_000_000)
142_000_000


## Clean Data

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [15]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [16]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [17]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'RANK', 'COLOR'],
      dtype='object')

In [18]:
pd.set_option('display.max_columns',None)

In [19]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
47,PAST SALE,2024-07-02,Condo/Co-op,100 S Pointe Dr Unit 2801/2802,Miami Beach,FL,33139.0,16150000.0,4.0,4.5,CONTINUUM ON SOUTH BEACH,3326.0,NaN,2002.0,NaN,4856.0,6911.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11500027,N,Y,25.767161,-80.132776,1,orange


In [21]:
def popup_html(row):
    building_name = row['LOCATION']
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [22]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [23]:
m.save('index.html')

## Data snagger

In [24]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [25]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
47,PAST SALE,2024-07-02,Condo/Co-op,100 S Pointe Dr Unit 2801/2802,Miami Beach,FL,33139.0,16150000.0,4.0,4.5,CONTINUUM ON SOUTH BEACH,3326.0,NaN,2002.0,NaN,4856.0,6911.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11500027,N,Y,25.767161,-80.132776,1,orange


In [ ]:
# , {df_top_ten[df_top_ten['PRICE'].idxmax()]['ADDRESS']} | Price ${df_top_ten[df_top_ten['PRICE'].idxmax()]['PRICE']:,.0f}

In [26]:
df_top_ten['PRICE'].iloc[0]

16150000.0

In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [27]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_07082024


# CREATE TEMPLATE 

In [28]:
muni_set = set(df_top_ten['CITY'])

In [29]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [30]:
df_top_ten.reset_index(inplace=True,drop=True)

In [31]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [32]:
top_sale

'CONTINUUM ON SOUTH BEACH at 100 S Pointe Dr Unit 2801/2802 in Miami Beach'

In [33]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [34]:
# df_top_ten

## RESUME HERE

In [35]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops Miami-Dade weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m Miami-Dade County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m Miami-Dade County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #MiamiDade condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, Miami-Dade County, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

Miami-Dade County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} sales average from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: CONTINUUM ON SOUTH BEACH closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,225,000M to $16,150,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Miami Beach, Key Biscayne, Sunny Isles Beach, Miami

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 118 condo sales totaling $92,157,794 million from Jul. 1st to Jul. 8th. The previous week, brokers closed 147 condo sales totaling $142,000,000.

Last week’s units sold for an average of $780,998, higher than the $748,529 sales average from the previous week. The average

In [36]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [37]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [38]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf")

print(story_checklist)


HED: CONTINUUM ON SOUTH BEACH closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,225,000M to $16,150,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Miami Beach, Key Biscayne, Sunny Isles Beach, Miami

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 118 condo sales totaling $92,157,794 million from Jul. 1st to Jul. 8th. The previous week, brokers closed 147 condo sales totaling $142,000,000.

Last week’s units sold for an average of $780,998, higher than the $748,529 sales average from the previous week. The average

In [39]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Miami-Beach/100-S-Pointe-Dr-33139/unit-2802/home/43281603


In [51]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Sunny-Isles-Beach/17749-Collins-Ave-33160/unit-1401/home/110130515


In [41]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Sunny-Isles-Beach/16699-Collins-Ave-33160/unit-2303/home/43368975


In [42]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Key-Biscayne/201-Crandon-Blvd-33149/unit-129/home/42921174


In [52]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami-Beach/100-S-Pointe-Dr-33139/unit-2802/home/43281603


In [53]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Miami/480-NE-31st-St-33137/unit-1101/home/168009143


In [48]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-07-02,Condo/Co-op,100 S Pointe Dr Unit 2801/2802,Miami Beach,FL,33139.0,16150000.0,4.0,4.5,CONTINUUM ON SOUTH BEACH,3326.0,NaN,2002.0,NaN,4856.0,6911.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/100-S-Po...,MARMLS,A11500027,N,Y,25.767161,-80.132776,1,orange,CONTINUUM ON SOUTH BEACH at 100 S Pointe Dr Un...
1,PAST SALE,2024-07-02,Condo/Co-op,17749 Collins Ave #1401,Sunny Isles Beach,FL,33160.0,6990000.0,3.0,6.5,Mansions at Acqualina,4609.0,NaN,2015.0,NaN,1517.0,10300.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11601281,N,Y,25.940065,-80.120264,2,blue,Mansions at Acqualina at 17749 Collins Ave #14...
2,PAST SALE,2024-07-01,Condo/Co-op,1643 Brickell Ave Unit TH2901,Miami,FL,33129.0,6650000.0,5.0,5.5,SANTA MARIA,5730.0,NaN,1997.0,NaN,1161.0,6516.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1643-Brickell-...,MARMLS,A11565486,N,Y,25.755388,-80.194955,3,blue,SANTA MARIA at 1643 Brickell Ave Unit TH2901 i...
3,PAST SALE,2024-07-02,Condo/Co-op,1643 Brickell Ave #3104,Miami,FL,33129.0,3925000.0,3.0,5.5,SANTA MARIA CONDO,4030.0,NaN,1997.0,NaN,974.0,9184.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1643-Brickell-...,MARMLS,A11614089,N,Y,25.755513,-80.195018,4,blue,SANTA MARIA CONDO at 1643 Brickell Ave #3104 i...
4,PAST SALE,2024-07-01,Condo/Co-op,901 Brickell Key Blvd #3308,Miami,FL,33131.0,3100000.0,4.0,4.5,CARBONELL CONDO,2860.0,NaN,2005.0,NaN,1084.0,3126.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/901-Brickell-K...,MARMLS,A11545665,N,Y,25.768991,-80.186892,5,blue,CARBONELL CONDO at 901 Brickell Key Blvd #3308...
5,PAST SALE,2024-07-03,Condo/Co-op,2127 Brickell Ave #1902,Miami,FL,33129.0,2515000.0,2.0,3.0,BRISTOL TOWER CONDO,2320.0,NaN,1993.0,NaN,1084.0,3912.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2127-Brickell-...,MARMLS,A11517058,N,Y,25.752160,-80.199342,6,blue,BRISTOL TOWER CONDO at 2127 Brickell Ave #1902...
6,PAST SALE,2024-07-01,Condo/Co-op,18671 Collins Ave #2401,Sunny Isles Beach,FL,33160.0,2100000.0,2.0,2.5,MILLENNIUM CONDO,2210.0,NaN,1999.0,NaN,950.0,1936.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11401903,N,Y,25.948699,-80.119756,7,blue,MILLENNIUM CONDO at 18671 Collins Ave #2401 in...
7,PAST SALE,2024-07-01,Condo/Co-op,480 NE 31st St #1101,Miami,FL,33137.0,1700000.0,4.0,4.5,GRAN PARAISO CONDO,2232.0,NaN,2019.0,NaN,762.0,2518.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/480-NE-31st-St...,MARMLS,A11519633,N,Y,25.806186,-80.187176,8,blue,GRAN PARAISO CONDO at 480 NE 31st St #1101 in ...
8,PAST SALE,2024-07-01,Condo/Co-op,201 Crandon Blvd #129,Key Biscayne,FL,33149.0,1525000.0,2.0,3.0,KEY COLONY NO 1,1809.0,NaN,1978.0,NaN,843.0,1949.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/201-Cra...,MARMLS,A11570146,N,Y,25.696814,-80.158094,9,blue,KEY COLONY NO 1 at 201 Crandon Blvd #129 in Ke...
9,PAST SALE,2024-07-01,Condo/Co-op,16699 Collins Ave #2303,Sunny Isles Beach,FL,33160.0,1225000.0,2.0,2.5,LA PERLA CONDO,1602.0,NaN,2006.0,NaN,765.0,1380.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11550405,N,Y,25.930187,-80.121374,10,blue,LA PERLA CONDO at 16699 Collins Ave #2303 in S...


## Time on Market Calculator

In [50]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 5, 26) ## List (Earlier) date
date2 = datetime(2024, 7, 2) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

37
